# Complexity Level 3

For CL3, there will need to be many additions/changes from CL1. List of changed: match making, liking with a message, "taking" (in the search function), messages die out after eight days, additional attributes such as education, job, ethnicity, and coins. 

Match making DONE
Men get paired with 21 women. They then decide to like or dislike based on their features.

The women get paired with 7 men, who have already liked them.

The function potential_match() will need to be split up into two functions, a potential match for men and a potential match for women. Additionally, the men will have to go through their likes before the females can go through theirs. The men will decide to like or dislike the 21 that they are given, and then based on those people that they like/dislike, the women will get a random selection of 7 of the men that like them. 


Attributes
Attributes women value more than men:
education, finances

Attribtues men value more than women:
Attractiveness

Attribtues men and women value the same:
age
ethnicity

Actions:
Tinder
Like with message
Like

Search route:
Search and like, but costs money

In [1]:
# importing appropriate packages
import networkx as nx #This has to be networkx v1.x not 2.x, but small modifications should allow for verion 2.x
import random
import numpy as np
import time
import CL3_V4a_dating_functions as df
import pandas as pd
from datetime import datetime 

# define the class
class complexity_level_3:
    def entry_method(self, size,
                     sim_times,amount_of_days,avg_swipes_m,avg_swipes_f,   
                     age_range, ages,
                     starting_coins,cost,
                     prob_males,
                     prob_m_likes,prob_f_likes,
                     probability):
        #imp is initial message probability
        origin_network = nx.erdos_renyi_graph(0, probability, directed=True,  seed=1234567) #setting a seed to ensure reproducability
        #We start off with a network called "Origin Network" that's basically an ER model. It is Directed
        #with a set initial size and probability of edge formation.
        
        
        #----------------------------------Initializing the tracing variables-------------------------#
        # variable "runs" does not do anything. It is from Dr. Shaheen's code. We do not use it.
        runs = 0
        
        
        
        # initialize lists which grab the averages at the end of each simulation
        # they populate by running the simulation multiple times by changing the "sim_times" variable
        # populating avgerage amount of matches, lists, dislikes, males, females, and messages
        total_match_list = list()
        total_like_list = list()
        total_dislike_list = list()
        total_male_list = list()
        total_female_list = list()
        total_message_list = list()
        
        # creating temporary variables to reset the values
        prob_m_likes_reset = prob_m_likes.copy()
        prob_f_likes_reset = prob_f_likes.copy()
        
        
        # starting the simulation
        for m in range(sim_times):
            print(m)
            
            #----------------------------------Initializing the probabilities types for the network-------------------------#
            # agents can be male or female
            gender_options = ["Male","Female"]
            
            # the amount of males and females differ, so their probabilties are defined here which are
            # used when initializing our network 
            gender_probabilities = [prob_males, 1-prob_males]
            
            
            
            
            
            # female and male lists for finding a partner
            # this list helps with the "potential_swipe" function in the dating functions file
            female_list = []
            male_list = []



            #----------------------------------Initializing the node attributes of the network-------------------------#
            # this for loop sets the node attributes
            for i in range(size):
                
                
                # setting the gender
                gender_randomized = np.random.choice(gender_options, p = gender_probabilities)
                
                
                # populating lists of males and females
                if gender_randomized == "Male":
                    # using the male probabilities to make a choice on male orientation
                    male_list.append(i)
                else:
                    # using the female probabilities to make a choice on female orientation
                    female_list.append(i)

                # adding the node to the network along with other random values for attribtues.
                origin_network.add_node(i, 
                                        {

                                            'gender': gender_randomized,
                                            'physical_attractiveness':random.random(),
                                            'age': np.random.randint(19,ages),
                                            'ethnicity': np.random.randint(1,5),         # 1:5 different races
                                            'education': np.random.randint(1,4),         # 1:4 different eduction levels
                                            'profession': np.random.randint(1,12),       # amount of different professions
                                            'coins': starting_coins                                # amount of coins to start
                                        })



            #----------------------------------Initializing the edge attributes of the network-------------------------#
            # setting the edge attributes
            # the edge attribute will have all the different types of actions that can be associated with it
            # it also has match_power and time
            # type and time_order were made by Dr. Shaheen's model. Not going to remove them.
            for edges in origin_network.edges():
                nx.set_edge_attributes(origin_network,'Time', 0)
                nx.set_edge_attributes(origin_network, 'Type', "ER")
                nx.set_edge_attributes(origin_network, 'time_order', 0)
                nx.set_edge_attributes(origin_network, 'message', 0 )
                nx.set_edge_attributes(origin_network, 'like', 0)
                nx.set_edge_attributes(origin_network, 'dislike', 0)
                nx.set_edge_attributes(origin_network, 'match', 0)
                nx.set_edge_attributes(origin_network, 'match_power',0)
            
            # name and time_count were in Dr. Shaheen's code. We do not use them.
            name = size + 1
            time_count= size + 1
            # "Time order, and time will be used for visualization techniques later" - Dr. Shaheen
            
            #------------------------------end of intialization-------------------------------------------------#


            #---------------------------------Initializing Constants/ empty lists--------------------------------------------#

            # initializing counts to count the actions as they happen
            match = 0
            male_like_count = 0
            female_like_count = 0
            male_like_w_message_count = 0 # counts the number of likes with messages made by male nodes
            female_like_w_message_count = 0 # counts the number of likes with messages made by female nodes
            dislikes = 0
            
            # need to establish an empty list for like_record and mess_like_record
                # after someone has liked someone, the edge will appear here for match purposes
            like_record = list()
            mess_like_record = list()


            # establish a dislike record to ensure that the individuals do not try and interact a second 
            # time if they have already passed on each other. This will ensure that we do not repeat.
            dislike_record = list()
            
            # reseting the values for liking
            prob_m_likes = prob_m_likes_reset.copy()#[0.6,0.8,0.05,0.10]
            prob_f_likes = prob_f_likes_reset.copy()#[0.6,0.8,0.03,0.06]
            
            
            # creating an empty list for match_records. This is created because within our match records, we need to see who has matched with who.
            match_records = list()
            
            
            
            # initialize list score record. This is used to keep track of all the like scores so that when messaging occurs,
            # we can divide it between 3 percentiles: low, med, high where the low has the lowest % chance of messaging and the high
            # has the greatest % chance of messaging
            list_score_m = []
            list_score_f = []


            # grabbing the node attributes for later usage in the code
            # essentially, using this code allows us to grab the gender,
            # attractiveness, ect. later in the code
            gender = nx.get_node_attributes(origin_network,'gender')
            physical_attractiveness = nx.get_node_attributes(origin_network,'physical_attractiveness')
            age = nx.get_node_attributes(origin_network,'age')
            ethnicity = nx.get_node_attributes(origin_network,'ethnicity')
            coins = nx.get_node_attributes(origin_network,'coins')
            # coins does not stay constant, so we cannot grab that attribute here. Instead, we have to grab it every time we want it

            #----------------------------------Starting the simulation-------------------------#

            for i in range(amount_of_days):
                print("\n\n -------------------------------------------------------- \n day #{}".format(i+1))
                

                # need to divide male and female swipes because the males need to like the females before they can find a match.
                # for each day, the males will, on average, have an opportiunity to like avg_swipes_m (number) of women
                # the women will, on average, have an opportunity to like avg_swipes_w (number) of women
                    # *** as long as they get a like from a male first *** #

                # ----------------------------------------------Starting Male Swipes------------------------------------------------------------
                # first male swipes




                # using the action that node_a decided upon




                # swiping action where males get paired up with females and they decide to like/dislike
                # the typical male does an action avg_swipes_m amount of times towards another female
                for swipe_number in range(avg_swipes_m*len(male_list)):


                    node_a = random.choice(male_list)





                    # getting the node attributes for node_a 
                    gender_a = gender[node_a]
                    age_a = age[node_a]
                    physical_attractiveness_a = physical_attractiveness[node_a]
                    ethnicity_a = ethnicity[node_a]
                    coins_a = coins[node_a]

                    # decide on an action
                    action = df.decide(coins_a,cost)
                    if action == "Swipe":
                        # grabbing a potentail mate for the male
                        node_b = df.potential_swipe_for_male(node_a,
                                                             gender_a, age_a,
                                                             male_list,female_list,
                                                             origin_network, age_range,
                                                             like_record, dislike_record)

                        # getting attributes for b
                        gender_b = gender[node_b]
                        age_b = age[node_b]
                        physical_attractiveness_b = physical_attractiveness[node_b]
                        ethnicity_b = ethnicity[node_b]
                        
                        
                        
                        # continuing based on too many fails in the function potential_swipe()
                        # essentially, if that node has already swiped on all of the opposite gender people in the area,
                        # then there will be no more people that agent_a can match with, and therefor we have to go to 
                        # the beginning of the swiping for loop.
                        if node_b == False:
                            # print("to many failed attempts")
                            continue


                        # retreive the like score between the two nodes
                        origin_network, like_a, like_b, list_score_m, list_score_f = df.match_power(node_a,gender_a, age_a,physical_attractiveness_a,ethnicity_a,
                                                                                       node_b,gender_b, age_b, physical_attractiveness_b,ethnicity_b,                                        
                                                                                       origin_network,
                                                                                       runs,i,
                                                                                       list_score_m,list_score_f)


        #                 print("The like score for {}, who is a {} is {}\n\n".format(node_a,gender_a,like_a))


                        origin_network, like_record, dislike_record, mess_like_record,male_like_count, female_like_count,male_like_w_message_count,female_like_w_message_count,coins_a,dislikes = df.Action2(node_a, coins_a,cost, node_b, gender_a, gender_b, 
                                                                                                                   like_a, like_b, physical_attractiveness_a,
                                                                                                                   physical_attractiveness_b,origin_network,
                                                                                                                   like_record,dislike_record,mess_like_record, 
                                                                                                                   runs,i,male_like_count,female_like_count,male_like_w_message_count,female_like_w_message_count,prob_m_likes.copy(),prob_f_likes.copy(),dislikes)



                    else:

                        coins_a,like_record, node_b, gender_b, age_b,physical_attractiveness_b,ethnicity_b, origin_network,female_like_count, male_like_count,like_a, like_b, list_score_m, list_score_f = df.search(cost,node_a, gender_a, age_a,coins_a,
                                                                                               physical_attractiveness_a,ethnicity_a,
                                                                                               physical_attractiveness, ethnicity, gender,age,
                                                                                               origin_network,runs,i,list_score_m,list_score_f,
                                                                                               male_list, female_list, age_range,like_record, dislike_record,
                                                                                               female_like_count, male_like_count)



                    # now to test for matches (after we have swiped/browsed)
                    # if the list [node b, node a] are in the like_record list, and the other way around is in the 
                    # like_record list, then that means that there is a match!
        #                 print("[node_b, node_a] in like_record: {} \n [node_a,node_b] in like_record: {}".format([node_b, node_a] in like_record,[node_a,node_b] in like_record))
                    if (([node_b, node_a] in like_record) and ([node_a,node_b] in like_record) and ([node_a,node_b] not in match_records) and ([node_b,node_a] not in match_records)):

#                         print("A match was found!!!")

                        origin_network, match_records, match = df.Match(node_a,node_b,gender_a, age_a, 
                                                                        age_b,physical_attractiveness_a,physical_attractiveness_b,
                                                                        origin_network,match_records, runs, 
                                                                        i, match)


                # ----------------------------------------------Ending Male Swipes------------------------------------------------------------



                # ----------------------------------------------Starting Female Swipes------------------------------------------------------------
                # now for female swipes/browsing
                # the typical female does an action avg_swipes_m amount of times towards another male

                for swipe_number in range(avg_swipes_f*len(female_list)):

                    # now onto female swipes




                    # using the action that node_a decided upon






                    # geab a random female
                    node_a = random.choice(female_list)





                    # getting the node attributes for node_a 
                    gender_a = gender[node_a]
                    age_a = age[node_a]
                    physical_attractiveness_a = physical_attractiveness[node_a]
                    ethnicity_a = ethnicity[node_a]
                    coins_a = coins[node_a]

                    # decide on an action
                    action = df.decide(coins_a,cost)

                    # swiping action where females get paired with males who have already liked them
                    if action == "Swipe":

                        # grabbing a potential male for the female who has liked the female already
                        node_b = df.potential_swipe_for_female(node_a,
                                                         gender_a, age_a,
                                                         male_list,female_list,
                                                         origin_network, age_range,
                                                         like_record, dislike_record)

                        # getting attributes for b
                        gender_b = gender[node_b]
                        age_b = age[node_b]
                        physical_attractiveness_b = physical_attractiveness[node_b]
                        ethnicity_b = ethnicity[node_b]

                        if node_b == False:
                            # print("to many failed attempts")
                            continue


                        # retreive the like score between the two nodes
                        origin_network, like_a, like_b, list_score_m, list_score_f = df.match_power(node_a,gender_a, age_a,physical_attractiveness_a,ethnicity_a,
                                                                                       node_b,gender_b, age_b, physical_attractiveness_b,ethnicity_b,                                     
                                                                                       origin_network,
                                                                                       runs,i,
                                                                                       list_score_m,list_score_f)


        #                 print("The like score for {}, who is a {} is {}\n\n".format(node_a,gender_a,like_a))


                        origin_network, like_record, dislike_record, mess_like_record,male_like_count, female_like_count,male_like_w_message_count,female_like_w_message_count,coins_a,dislikes = df.Action2(node_a, coins_a,cost, 
                        node_b, gender_a, gender_b, 
                        like_a, like_b, physical_attractiveness_a,
                        physical_attractiveness_b,origin_network,
                        like_record,dislike_record,mess_like_record, 
                        runs,i,male_like_count,female_like_count,male_like_w_message_count,female_like_w_message_count,
                        prob_m_likes.copy(),prob_f_likes.copy(),dislikes)
            
            
            
                    else:
                        coins_a,like_record, node_b, gender_b, age_b,physical_attractiveness_b,ethnicity_b,origin_network,female_like_count, male_like_count,like_a, like_b, list_score_m, list_score_f = df.search(cost,node_a, gender_a, age_a,coins_a,
                        physical_attractiveness_a,ethnicity_a,
                        physical_attractiveness, ethnicity,gender,age,
                        origin_network,runs,i,list_score_m,list_score_f,
                        male_list, female_list, age_range,like_record, dislike_record,
                        female_like_count, male_like_count)



                    # now to test for matches (after we have swiped/browsed)
                    # if the list [node b, node a] are in the like_record list, and the other way around is in the 
                    # like_record list, then that means that there is a match!
    #                 print("[node_b, node_a] in like_record: {} \n [node_a,node_b] in like_record: {}".format([node_b, node_a] in like_record,[node_a,node_b] in like_record))
                    if (([node_b, node_a] in like_record) and ([node_a,node_b] in like_record) and ([node_a,node_b] not in match_records) and ([node_b,node_a] not in match_records)):

#                         print("A match was found!!!")

                        origin_network, match_records, match = df.Match(node_a,node_b,gender_a, age_a, 
                                                                        age_b,physical_attractiveness_a,physical_attractiveness_b,
                                                                        origin_network,match_records, runs, 
                                                                        i, match)



    #         nx.write_gexf(origin_network, "sim_" + str(sim_number) + "run" + str("___") + "iter" + str(time.time()) + ".gexf")

            # now for female swipes
    #         print("The amount of matches: {}".format(match))
    
        

            # ----------------------------------------------Starting Messages Now------------------------------------------------------------
            print("\n\n\n\n\n")
            print("The amount of matches: {}".format(match))
            
            print("Message Block is starting now")

            origin_network, message_count, message_list = df.message(origin_network,match,list_score_m,list_score_f,runs,i,match_records,gender,age,physical_attractiveness)

            print("there are {} Males and {} Females".format(len(male_list),len(female_list)))
            print("the males liked the females a total of {} times \nthe females liked the males a total of {} \nthe males liked the females with message a total of {} times \nthe females with a message liked the males a total of {}".format(male_like_count,female_like_count,male_like_w_message_count,female_like_w_message_count))


            print("there are {} in the like record and {} in the dislike record.".format(len(like_record),len(dislike_record)))
    #         print("Like record ",like_record)
#             print("The message count is {} through {} iterations".format(message_count,j))
            #nx.write_gexf(origin_network, "p.gexf")
                
            total_match_list.append(match)
            total_like_list.append(len(like_record))
            total_dislike_list.append(dislikes)
            total_male_list.append(len(male_list))
            total_female_list.append(len(female_list))
            total_message_list.append(message_count)
                
        zippedList =  list(zip(total_match_list, total_like_list, total_dislike_list, total_male_list, total_female_list, total_message_list))
        counts_df = pd.DataFrame(zippedList, columns = ['matches' , 'likes', 'dislikes','males','females','messages']) 
        
        
        return origin_network, counts_df




In [ ]:
# running the model and setting the parameters
if __name__ == '__main__':
    
    
    size = 1000 # Initial size of the network
    sim_times = 25 # simulation number
    amount_of_days = 3 # defines how many days you want the simulation to run
    avg_swipes_m = 21 # average amount of male swipes between each node per day
    avg_swipes_f = 5 # average amount of female swipes between each node per day
    age_range = 10 # +/- this age for potential swipes/matches
    ages = 40 # number of ages 
    starting_coins = 1010 # the number of coins that you want each agent to start with
    cost = 385 # cost of a like when searching
    prob_males = 0.68 # proportion of those in the population who are males
    Simulation_times = 1 # how many times would you like to run the simulation
    probability = 0.000 # intializing probability of the erdos renyi network    /// should keep this at 0 for our simulations
    
    
    # typical default values: 0.6,0.8,0.05,0.1
    high_like_male_prob = 0.6 # if the male is greatly attracted to the female, this is the probability that the male will like
    high_like_w_message_male_prob = 0.2 # if the male is greatly attracted to the female, this is the probability that the male will like with a message
    low_like_male_prob = 0.05 # if the male is NOT greatly attracted to the female, this is the probability that the male will like
    low_like_w_message_male_prob = 0.07 # if the male is NOT greatly attracted to the female, this is the probability that the male will like with a message
    
    # typical default values: [0.6,0.8,0.03,0.06]
    high_like_female_prob = 0.6 # if the female is greatly attracted to the male, this is the probability that the male will like
    high_like_w_message_female_prob = 0.2 # if the female is greatly attracted to the male, this is the probability that the female will like with a message
    low_like_female_prob = 0.03 # if the female is NOT greatly attracted to the male, this is the probability that the male will like
    low_like_w_message_female_prob = 0.05 # if the female is NOT greatly attracted to the male, this is the probability that the female will like with a message
    
    # adding the probabilities into the list for the function: Action2
    prob_m_likes = [high_like_male_prob,high_like_male_prob + high_like_w_message_male_prob,
                    low_like_male_prob,low_like_w_message_male_prob+low_like_male_prob]
    
    prob_f_likes = [high_like_female_prob,high_like_w_message_female_prob+high_like_female_prob,
                    low_like_female_prob,low_like_w_message_female_prob+low_like_female_prob]
    
    global max_size
    max_size = 20 # set this value to set all other relevant parameters. The model will ouput a network roughly at this size
#     entry_iterations = int(amount_of_days * 0.3) # used to iterate actions for entry only.
n = 1

# self, size, probability, max_swipes, prob_female_likes, prob_female_super_likes, prob_male_likes, prob_male_super_likes, prob_males, prob_ori_m, prob_ori_f):

while n <= Simulation_times:
    print ("simulation starts...")
    A = complexity_level_3() #iterations is always set at 2 for now
    B, count_df = A.entry_method(size,
                           sim_times,amount_of_days,avg_swipes_m,avg_swipes_f,   
                           age_range, ages,
                           starting_coins,cost,
                           prob_males,
                           prob_m_likes,prob_f_likes,
                           probability)
    
    

    print("Simulation Concluded")
    n += 1
    
    
    

simulation starts...
0


 -------------------------------------------------------- 
 day #1


 -------------------------------------------------------- 
 day #2


 -------------------------------------------------------- 
 day #3






The amount of matches: 0
Message Block is starting now
there are 678 Males and 322 Females
the males liked the females a total of 2592 times 
the females liked the males a total of 595 
the males liked the females with message a total of 641 times 
the females with a message liked the males a total of 640
there are 4468 in the like record and 31849 in the dislike record.
1


 -------------------------------------------------------- 
 day #1


 -------------------------------------------------------- 
 day #2


 -------------------------------------------------------- 
 day #3






The amount of matches: 0
Message Block is starting now
there are 683 Males and 317 Females
the males liked the females a total of 2497 times 
the females liked the males a tot

# Exporting the files

In [ ]:
# telling the time
today = datetime.now()
month = today.month
day = today.day
year = today.year
hour = today.hour
minute = today.minute


# creating the parameters_df
parameters_used = ["complexity_level","date","time","sim_times", "amount_of_days", "avg_swipes_m", "avg_swipes_f", "age_range", "age","high_like_male_prob","low_like_male_prob","high_like_female_prob","low_like_female_prob"]
parameter_values = [["3","{}/{}/{}".format(month,day,year),"{}:{}".format(hour,minute),sim_times, amount_of_days, avg_swipes_m, avg_swipes_f, age_range, ages + 1, high_like_male_prob, low_like_male_prob, high_like_female_prob, low_like_female_prob]]
parameters_df = pd.DataFrame(parameter_values, columns = parameters_used)


# exportint the parameters and the count_df
parameters_df.to_csv('CL3 DATE {} MON {} DAY {} YEAR TIME {} HR {} MIN PARAMETERS.csv'.format(month,day,year,hour,minute))
count_df.to_csv("CL3 DATE {} MON {} DAY {} YEAR TIME {} HR {} MIN COUNTS.csv".format(month,day,year,hour,minute))

# exporting the last network of the simulation itself


# Exporting the network

In [ ]:
# to visualize in r
nx.write_edgelist(B,'test.csv',data=['like','dislike','match','message','match_power'])
# need to read the dataframe back in 
edge_list_df = pd.read_csv("test.csv",names=["node_a","node_b","like","dislike",'match','message','match_power'],sep=' ')
# B.get_edge_data()
# initialize the lists to keep track of genders 
gender_list = list()
physical_attractiveness_list = list()
age_list = list()
ethnicity_list = list()

# be able to retreive the genders,attractiveness,ages, and the ethnicities of the nodes
gender = nx.get_node_attributes(B,'gender')
physical_attractiveness = nx.get_node_attributes(B,'physical_attractiveness')
age = nx.get_node_attributes(B,'age')
ethnicity = nx.get_node_attributes(B,'ethnicity')


for node_a in edge_list_df['node_a']:
    
    
    
    
    gender_list.append(gender[node_a])
    physical_attractiveness_list.append(physical_attractiveness[node_a])
    age_list.append(age[node_a])
    ethnicity_list.append(ethnicity[node_a])
    
    
node_a_list = edge_list_df["node_a"].to_list()
node_b_list = edge_list_df["node_b"].to_list()
like_list = edge_list_df["like"].to_list()
dislike_list = edge_list_df["dislike"].to_list()
match_list = edge_list_df["match"].to_list()
message_list = edge_list_df["message"].to_list()
match_power_list = edge_list_df["match_power"].to_list()
zipped_list = list(zip(node_a_list,node_b_list,like_list,dislike_list,match_list,message_list,match_power_list,gender_list,physical_attractiveness_list,age_list,ethnicity_list))
edge_att_df = pd.DataFrame(zipped_list, columns =['node_a', 'node_b','like','dislike','match','message','match_power','gender_node_a','physical_attractiveness_node_a','age_node_a','ethnicity_node_a'])
edge_att_df.to_csv("CL3 DATE {}M {}D {}Y TIME {}hr {}min NETWORK.csv".format(month,day,year,hour,minute))
print("done")